# Notebook for visualizing and exploring the data

## Working the CSVs and Excel sheets

In [1]:
# Exploring the 'training' dataset
import pandas as pd
df = pd.read_csv("txt2sql_alerce_train_v2.csv")
df.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\n-- mjd date for December = 59914.0\n-- mjd d...,\n-- Super Nova (SNe) is a large explosion tha...,"\nSELECT\n object.oid, probability.class_na...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\nSELECT\n sq1.oid, sq1.probability as SN_pro...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\n-- mjd date for September 01 = 60188.0\n-- m...,\n-- A fast riser is defined as an object whos...,"\nSELECT\n sq.oid, sq.probability, sq.candi...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,0,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\n-- mjd date for February 01 = 59976.0\n-- mj...,0,"\nWITH catalog ( source_id, ra, dec) AS (\n ...",advanced,spatial,simple,NaN,NaN,NaN


In [2]:
for request in df["request"]:
    if "SQL" in request:
        print(f"{df[df['request'] == request]['req_id'].values[0]}: {request}")

32: Given a list of oids of asteroids candidates, write a SQL script that returns data from the ss_ztf table with a positive ssdistnr value.
20: Query objects classified by the light curve classifier that are most likely to be RRL with a probability larger than 0.55 and that have 100 or more detections. Write in PostgresSQL.
18: Write a script in PostgreSQL with a query asking for all the probabilities for all the sources classfied as either AGN, QSO or Blazar by the light curve classifier. Define a minimum number of detections of 40, and a minimum time span between the first and the last detection of 200 days, in order to ensure the detection of a variable signal from the light curves. Also, filter out sources using a predicted class probability higher than 0.8 for the Blazar class and 0.95 for the AGN and QSO classes. Collect similar numbers for each class using a UNION statement between each sub-query defined by each probability. The query must return the columns oid, meanra, meande

In [10]:
print(df[df["request"] == "Find at most 50 ZTF objects that satisfy the following conditions: have a probability larger than 0.7 of being an AGN in the light curve classifier version 'hierarchical_rf_1.1.0', a distance to the nearest PS1 source lower than 2 arcsec, and a difference between the star/galaxy score from PS1 and the star/galaxy score from SExtractor lower than 0.5. Return the following columns. For the ZTF object: ZTF identifier and probability; for each epoch: candidate and filter identifier, reduced chi-square, star/galaxy score from SExtractor, and also the identifier, PS1 star/galaxy score and distance of the nearest PS1 source"]["gold_query"][46])

SELECT
    sq1.oid,
    sq1.probability, sq1.candid, sq1.fid,
    sq1.chipsf, sq1.classtar,
    sq2.objectidps1, sq2.sgscore1,
    sq2.distpsnr1
FROM
(
SELECT
    probability.oid,
    probability.probability, dataquality.candid,
    dataquality.fid, dataquality.chipsf,
    dataquality.classtar
FROM
    probability INNER JOIN
    dataquality
    ON probability.oid = dataquality.oid
WHERE
    classifier_name = 'lc_classifier'
    AND classifier_version = 'hierarchical_rf_1.1.0'
    AND class_name = 'AGN'
    AND probability > 0.7
) AS sq1
INNER JOIN
(
SELECT
    probability.oid, ps1_ztf.candid,
    ps1_ztf.objectidps1, ps1_ztf.sgscore1,
    ps1_ztf.distpsnr1
FROM
    probability INNER JOIN
    ps1_ztf
    ON probability.oid = ps1_ztf.oid
WHERE
    classifier_name = 'lc_classifier'
    AND classifier_version = 'hierarchical_rf_1.1.0'
    AND class_name = 'AGN'
    AND probability > 0.7
) AS sq2
ON sq1.oid = sq2.oid
AND sq1.candid = sq2.candid
WHERE
    sq2.distpsnr1 < 2
    AND sq2.sgscor

In [3]:
print(df[df["difficulty"] == "medium"]["request"][1])
print(df[df["difficulty"] == "medium"]["gold_query"][1])

Get the object identifiers and probabilities in the light curve classifier for objects classified in the light curve classifier as SNIa with ranking=1 and CV/Nova with ranking=2, where the difference between the probabilities at each ranking is lower than 0.1. Return oids, and the probability for each class

SELECT
  sq1.oid, sq1.probability as SN_prob, sq2.probability as CV_prob
FROM
  (
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name='SNIa'
    AND probability.ranking = 1
) AS sq1
  INNER JOIN (
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name='CV/Nova'
    AND probability.ranking = 2
) as sq2
  ON sq1.oid = sq2.oid
WHERE
  sq1.probability - sq2.probability < 0.1



In [3]:
print(df[df["difficulty"] == "advanced"]["request"][2])
print(df[df["difficulty"] == "advanced"]["gold_query"][2])

Get the object identifiers, probabilities in the stamp classifier and light curves (only detections) for objects whose highest probability in the stamp classifier is obtained for class SN, that had their first detection in the first 2 days of september, and that qualify as fast risers.

SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid as magstat_fid, magstat.dmdt_first
FROM
  (
SELECT
  detection.oid, detection.candid, detection.fid, detection.mjd,
  obj_oids.probability
FROM
  (
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='stamp_classifier'
    AND probability.class_name='SN'
    AND probability.ranking=1
    AND object.firstmjd > 60188.0
    AND object.firstmjd < 60189.0
) as obj_oids
    INNER JOIN
    detection ON detection.oid = obj_oids.oid
) AS sq
  INNER JOIN magstat
  ON sq.oid = magstat.oid
WHERE
  magstat.dmdt_first < -0.25
ORDE

In [6]:
# Reading the excel file
excel = pd.read_excel("SQLusecases_alerce.xlsx", 
                      sheet_name="examples_alerce_usecasesV3_1")

# Cleaning the excel a bit
excel = excel.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
excel.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,Set,python_format
0,0,Get objects that are likely to be YSOs (possib...,"['probability', 'feature']",\n-- feature.name can be 'Multiband_period'\n-...,\n-- Multiband_period: Period obtained using t...,"\nSELECT\n oid, probability, value, name, fid...",advanced,object,tree,Train,"sub_query_1='''\nSELECT\n feature.oid, prob_o..."
1,1,Get all the objects classified as AGN with a p...,"['object', 'probability', 'feature', 'magstat']",\n-- object.ndet represents the number of dete...,\n-- Amplitude: Half of the difference between...,"\nSELECT\n sq.oid, sq.value, sq.name, sq.fid ...",advanced,object,tree,Test,\nsub_query_object='''\nSELECT\n object.oid...
2,2,Give me the objects classified as YSO by their...,['probability'],NaN,NaN,"\nSELECT\n oid, probability\nFROM\n prob...",simple,object,none,Train,"query='''\nSELECT\n oid, probability\nFROM\..."
3,3,Give me the objects classified as YSO by the l...,"['object', 'probability']","\n-- last june in mjd date: [start=60096.0, en...",NaN,\nSELECT\n *\nFROM\n probability\nWHERE\...,simple,object,simple,Train,query=f'''\nSELECT\n *\nFROM\n probabili...
4,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,NaN,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,Train,# objects classified as SN II with probability...


In [17]:
# Getting the query we've been working on
mask = df[df["difficulty"] == "advanced"]["request"][2]
p_format = excel[excel["request"] == mask]["python_format"][15]
print(p_format)

sub_query_object=f'''
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='stamp_classifier'
    AND probability.class_name='SN'
    AND probability.ranking=1
    AND object.firstmjd > {mjd_first_16}
    AND object.firstmjd < {mjd_last_16}
'''

sub_query_detection=f'''
SELECT
  detection.oid, detection.candid, detection.fid, detection.mjd,
  obj_oids.probability
FROM
  ({sub_query_object}) as obj_oids
    INNER JOIN
    detection ON detection.oid = obj_oids.oid
'''

### detections
query=f'''
SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid as magstat_fid, magstat.dmdt_first
FROM
  ({sub_query_detection}) AS sq
  INNER JOIN magstat
  ON sq.oid = magstat.oid
WHERE
  magstat.dmdt_first < -0.25
ORDER BY oid
'''


In [9]:
# Getting the query we've been working on
mask = df[df["difficulty"] == "medium"]["request"][1]
p_format = excel[excel["request"] == mask]["python_format"][10]
print(p_format)

sub_query_object_1='''
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name='SNIa'
    AND probability.ranking = 1
'''

sub_query_object_2='''
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name='CV/Nova'
    AND probability.ranking = 2
'''

query=f'''
SELECT
  sq1.oid, sq1.probability as SN_prob, sq2.probability as CV_prob
FROM
  ({sub_query_object_1}) AS sq1
  INNER JOIN ({sub_query_object_2}) as sq2
  ON sq1.oid = sq2.oid
WHERE
  sq1.probability - sq2.probability < 0.1
'''


# Tests

### Query for tests:
### Get the object identifiers, probabilities in the stamp classifier and light curves (only detections) for objects whose highest probability in the stamp classifier is obtained for class SN, that had their first detection in the first 2 days of september, and that qualify as fast risers.

In [17]:
# Gold result table
gold_query = """ 
SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid as magstat_fid, magstat.dmdt_first
FROM
  (
SELECT
  detection.oid, detection.candid, detection.fid, detection.mjd,
  obj_oids.probability
FROM
  (
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='stamp_classifier'
    AND probability.class_name='SN'
    AND probability.ranking=1
    AND object.firstmjd > 60188.0
    AND object.firstmjd < 60189.0
) as obj_oids
    INNER JOIN
    detection ON detection.oid = obj_oids.oid
) AS sq
  INNER JOIN magstat
  ON sq.oid = magstat.oid
WHERE
  magstat.dmdt_first < -0.25
ORDER BY oid
"""

In [18]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(gold_query, con=engine)

,oid,probability,candid,fid,mjd,magstat_fid,dmdt_first
0,ZTF18aaiaclp,0.520409,2434223984915015031,2,60188.223982,2,-0.445361
1,ZTF18aakjgzj,0.319556,2434219693515015039,2,60188.219699,2,-0.341342
2,ZTF18aaleydm,0.613368,2434217801715015066,2,60188.217801,2,-0.830610
3,ZTF18aamxief,0.323881,2434223031815015040,2,60188.223032,2,-0.298997
4,ZTF18aanglcr,0.336435,2434215873315015004,2,60188.215880,2,-0.713267
5,ZTF18aaouvke,0.282244,2434220161815015099,2,60188.220162,2,-0.333054
6,ZTF18aaqsuyn,0.414416,2434215875015015115,2,60188.215880,2,-0.880376
7,ZTF18aattenu,0.599858,2434216851815015097,2,60188.216852,2,-0.651601
8,ZTF18aaxkcvg,0.492333,2434224440015015034,2,60188.224444,2,-0.346522
9,ZTF18abbinqq,0.497010,2434182954115015170,1,60188.182951,1,-0.529858


In [2]:
sub_query_object=f'''
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='stamp_classifier'
    AND probability.class_name='SN'
    AND probability.ranking=1
    AND object.firstmjd > 60188.0
    AND object.firstmjd < 60189.0
'''

sub_query_detection=f'''
SELECT
  detection.oid, detection.candid, detection.fid, detection.mjd,
  obj_oids.probability
FROM
  ({sub_query_object}) as obj_oids
    INNER JOIN
    detection ON detection.oid = obj_oids.oid
'''

### detections
query=f'''
SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid as magstat_fid, magstat.dmdt_first
FROM
  ({sub_query_detection}) AS sq
  INNER JOIN magstat
  ON sq.oid = magstat.oid
WHERE
  magstat.dmdt_first < -0.25
ORDER BY oid
'''

In [3]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(query, con=engine)

,oid,probability,candid,fid,mjd,magstat_fid,dmdt_first
0,ZTF18aaiaclp,0.520409,2434223984915015031,2,60188.223982,2,-0.445361
1,ZTF18aakjgzj,0.319556,2434219693515015039,2,60188.219699,2,-0.341342
2,ZTF18aaleydm,0.613368,2434217801715015066,2,60188.217801,2,-0.830610
3,ZTF18aamxief,0.323881,2434223031815015040,2,60188.223032,2,-0.298997
4,ZTF18aanglcr,0.336435,2434215873315015004,2,60188.215880,2,-0.713267
5,ZTF18aaouvke,0.282244,2434220161815015099,2,60188.220162,2,-0.333054
6,ZTF18aaqsuyn,0.414416,2434215875015015115,2,60188.215880,2,-0.880376
7,ZTF18aattenu,0.599858,2434216851815015097,2,60188.216852,2,-0.651601
8,ZTF18aaxkcvg,0.492333,2434224440015015034,2,60188.224444,2,-0.346522
9,ZTF18abbinqq,0.497010,2434182954115015170,1,60188.182951,1,-0.529858


## Test 1: 2024/10/10

In [8]:
# Step 2: Retrieve objects classified as 'SN' by the stamp classifier
sub_query_probabilities = """
SELECT oid, probability 
FROM probability 
WHERE class_name = 'SN' 
AND classifier_name = 'stamp_classifier' 
AND ranking = 1
"""

# Step 3: Filter objects with first detection in the first two days of September
sub_query_first_detection = """
SELECT oid 
FROM detection 
WHERE (timestamp '1970-01-01' + (mjd + 2400000.5 - 2440587.5) * INTERVAL '1 day')::date 
BETWEEN '2024-09-01' AND '2024-09-02'
AND isdiffpos = 1
"""

# Step 4: Identify objects qualifying as fast risers based on features
sub_query_fast_risers = """
SELECT oid 
FROM feature 
WHERE (name = 'dmdt_first' AND value > 0.5)  -- Example of fast rising criteria, adjust threshold as needed
AND (name = 'dm_first' AND value > 0.3)
AND (name = 'delta_mjd_fid' AND value < 2)
"""

# Step 5: Retrieve light curves (detections) for filtered objects
sub_query_light_curves = """
SELECT oid, mjd, magpsf 
FROM detection 
WHERE isdiffpos = 1
"""

# Step 6: Combine the results from subqueries
full_query = f"""
SELECT p.oid, p.probability, d.mjd, d.magpsf
FROM ({sub_query_probabilities}) p
INNER JOIN ({sub_query_first_detection}) fd ON p.oid = fd.oid
INNER JOIN ({sub_query_fast_risers}) fr ON p.oid = fr.oid
INNER JOIN ({sub_query_light_curves}) d ON p.oid = d.oid
"""

In [9]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

,oid,probability,mjd,magpsf


## Test 2: 2024/10/14

### GPT 4o

In [26]:
# Sub-query to retrieve object identifiers and probabilities for class 'SN' in the stamp classifier with ranking=1
sub_query_object = '''
SELECT
    object.oid, probability.probability
FROM
    object 
INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name = 'stamp_classifier'
    AND probability.class_name = 'SN'
    AND probability.ranking = 1
'''

# Sub-query to filter objects with detections in the first two days of September
# Use MJD conversion from standard date using the Julian reference date '1858-11-17'
sub_query_detection = f'''
SELECT
    detection.oid, detection.candid, detection.fid, detection.mjd,
    obj_oids.probability
FROM
    ({sub_query_object}) as obj_oids
INNER JOIN
    detection ON detection.oid = obj_oids.oid
WHERE
    detection.mjd >= (EXTRACT(EPOCH FROM (timestamp '2023-09-01' - timestamp '1858-11-17 00:00:00')) / 86400)
    AND detection.mjd < (EXTRACT(EPOCH FROM (timestamp '2023-09-03' - timestamp '1858-11-17 00:00:00')) / 86400)
    AND detection.isdiffpos = 1  -- only detections
'''

# Sub-query to filter fast risers based on features (e.g., dmdt_first indicating rapid changes in magnitude)
sub_query_fast_risers = '''
SELECT
    feature.oid
FROM
    feature
WHERE
    feature.name = 'dmdt_first'
    AND feature.value < -0.25  -- filter for fast risers
'''

# Final query to combine object, detection, and feature information
full_query = f'''
SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid as magstat_fid, magstat.dmdt_first
FROM
  ({sub_query_detection}) AS sq
INNER JOIN magstat
    ON sq.oid = magstat.oid
WHERE
    sq.oid IN ({sub_query_fast_risers})
ORDER BY sq.oid
'''


In [27]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

,oid,probability,candid,fid,mjd,magstat_fid,dmdt_first


### GPT4o1

In [28]:
# Subquery to select objects classified as 'SN' by the 'stamp_classifier' with ranking 1 and first detection in the first 2 days of September
sub_query_object_sn = '''
SELECT
    object.oid,
    probability.probability
FROM
    object
    INNER JOIN probability ON object.oid = probability.oid
WHERE
    probability.classifier_name = 'stamp_classifier'
    AND probability.class_name = 'SN'
    AND probability.ranking = 1
    AND object.firstmjd >= 60174
    AND object.firstmjd < 60176
'''

# Subquery to select objects qualifying as fast risers (feature 'dmdt_first' < -0.25)
sub_query_feature = '''
SELECT
    feature.oid
FROM
    feature
WHERE
    feature.name = 'dmdt_first'
    AND feature.value < -0.25
'''

# Subquery to get objects that are both classified as 'SN' and qualify as fast risers
sub_query_fast_risers = f'''
SELECT
    obj_sn.oid,
    obj_sn.probability
FROM
    ({sub_query_object_sn}) AS obj_sn
    INNER JOIN ({sub_query_feature}) AS feat ON obj_sn.oid = feat.oid
'''

# Subquery to retrieve detections (light curves) for the filtered objects
sub_query_detections = f'''
SELECT
    det.oid,
    det.candid,
    det.fid,
    det.mjd,
    det.magpsf,
    obj_fr.probability
FROM
    ({sub_query_fast_risers}) AS obj_fr
    INNER JOIN detection AS det ON obj_fr.oid = det.oid
WHERE
    det.isdiffpos = 1
'''

# Final query to select the required data
full_query = f'''
SELECT
    *
FROM
    ({sub_query_detections}) AS sq
ORDER BY
    sq.oid
'''

In [29]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

,oid,candid,fid,mjd,magpsf,probability


## Test 3: 2024/10/17

### GPT 4o. Try 1

In [25]:
sub_query_object = f'''
SELECT
    object.oid, probability.probability
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='stamp_classifier'
    AND probability.class_name='SN'
    AND probability.ranking=1
    AND object.firstmjd BETWEEN 60188.0 AND 60189.0
'''

sub_query_fast_risers = f'''
SELECT
    feature.oid
FROM
    feature
WHERE
    feature.name = 'delta_mag_fid'
    AND feature.value > -0.25
'''

sub_query_filtered_objects = f'''
SELECT
    obj_oids.oid, obj_oids.probability
FROM
    ({sub_query_object}) AS obj_oids
    INNER JOIN
    ({sub_query_fast_risers}) AS fast_risers
    ON obj_oids.oid = fast_risers.oid
'''

sub_query_detection = f'''
SELECT
    detection.oid, detection.candid, detection.fid, detection.mjd, detection.magpsf, detection.sigmapsf,
    filtered_oids.probability
FROM
    detection
    INNER JOIN
    ({sub_query_filtered_objects}) AS filtered_oids
    ON detection.oid = filtered_oids.oid
WHERE
    detection.isdiffpos = 1
'''

full_query = f'''
SELECT
    detection.oid, detection.mjd, detection.magpsf, detection.sigmapsf, detection.probability
FROM
    ({sub_query_detection}) AS detection
ORDER BY detection.oid
'''

In [26]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

,oid,mjd,magpsf,sigmapsf,probability
0,ZTF18adruhfi,60188.156227,18.769300,0.220319,0.316951
1,ZTF18adruhfi,60193.288970,17.287200,0.154040,0.316951
2,ZTF18adruhfi,60195.150243,18.771526,0.111644,0.316951
3,ZTF18adruhfi,60202.170926,18.901512,0.169160,0.316951
4,ZTF18adruhfi,60202.170926,18.901500,0.169160,0.316951
...,...,...,...,...,...
664,ZTF23aawmham,60470.350706,20.886000,0.307970,0.632489
665,ZTF23aawmham,60470.394086,20.440300,0.205180,0.632489
666,ZTF23aawmham,60472.415775,20.642900,0.239811,0.632489
667,ZTF23aawmham,60475.318137,20.744000,0.381222,0.632489


### GPT 4o. Try 2

In [35]:
# Step 1: Filter objects by first detection in the first two days of September
sub_query_object=f'''
SELECT oid 
FROM object 
WHERE firstmjd BETWEEN 60188.0 AND 60189.0
'''

# Step 2: Filter by highest probability for class 'SN' in stamp classifier
sub_query_probability=f'''
SELECT oid, probability 
FROM probability 
WHERE class_name = 'SN' 
  AND classifier_name = 'stamp_classifier' 
  AND ranking = 1
'''

# Step 3: Combine first detection and highest probability
sub_query_combined=f'''
SELECT obj.oid, prob.probability 
FROM ({sub_query_object}) as obj
INNER JOIN ({sub_query_probability}) as prob
USING (oid)
'''

# Step 4: Filter objects qualifying as fast risers based on the feature table with threshold
sub_query_fast_risers=f'''
SELECT oid 
FROM feature 
WHERE name = 'delta_mag_fid' 
  AND value > 0.5  -- Fast riser threshold
'''

# Step 5: Filter by fast riser criteria using INTERSECT (matching only oid columns)
sub_query_filtered=f'''
SELECT oid, probability 
FROM ({sub_query_combined}) as combined
INTERSECT
SELECT oid, NULL as probability 
FROM ({sub_query_fast_risers}) as risers
'''

# Step 6: Retrieve light curve data (detections only)
sub_query_detection=f'''
SELECT d.oid, d.mjd, d.magpsf, d.sigmapsf, filtered.probability
FROM detection d 
INNER JOIN ({sub_query_filtered}) as filtered
USING (oid)
WHERE d.isdiffpos = 1
'''

# Final query to retrieve object identifiers, probabilities, and light curve detections
full_query = f'''
SELECT d.oid, d.probability, d.mjd, d.magpsf, d.sigmapsf 
FROM ({sub_query_detection}) as d
'''


In [36]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

OperationalError: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
SELECT d.oid, d.probability, d.mjd, d.magpsf, d.sigmapsf 
FROM (
SELECT d.oid, d.mjd, d.magpsf, d.sigmapsf, filtered.probability
FROM detection d 
INNER JOIN (
SELECT oid, probability 
FROM (
SELECT obj.oid, prob.probability 
FROM (
SELECT oid 
FROM object 
WHERE firstmjd BETWEEN 60188.0 AND 60189.0
) as obj
INNER JOIN (
SELECT oid, probability 
FROM probability 
WHERE class_name = 'SN' 
  AND classifier_name = 'stamp_classifier' 
  AND ranking = 1
) as prob
USING (oid)
) as combined
INTERSECT
SELECT oid, NULL as probability 
FROM (
SELECT oid 
FROM feature 
WHERE name = 'delta_mag_fid' 
  AND value > 0.5  -- Fast riser threshold
) as risers
) as filtered
USING (oid)
WHERE d.isdiffpos = 1
) as d
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Test 4: 2024/10/18

In [1]:
# Step 1: Retrieve object identifiers and probabilities for objects classified as 'SN' in the stamp classifier with the highest probability
sub_query_object = f'''
SELECT
    object.oid, probability.probability
FROM
    object
INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name = 'stamp_classifier'
    AND probability.class_name = 'SN'
    AND probability.ranking = 1
    AND object.firstmjd BETWEEN 60188.0 AND 60189.0
'''

# Step 2: Retrieve detections for the filtered objects from Step 1
sub_query_detection = f'''
SELECT
    detection.oid, detection.candid, detection.fid, detection.mjd,
    obj_oids.probability
FROM
    ({sub_query_object}) as obj_oids
INNER JOIN
    detection
    ON detection.oid = obj_oids.oid
'''

# Step 3: Filter for fast risers using magstat table by joining it with the filtered detections from Step 2
query = f'''
SELECT
    sq.oid, sq.probability, sq.candid, sq.fid, sq.mjd,
    magstat.fid AS magstat_fid, magstat.dmdt_first
FROM
    ({sub_query_detection}) AS sq
INNER JOIN
    magstat
    ON sq.oid = magstat.oid
WHERE
    magstat.dmdt_first < -0.25
ORDER BY sq.oid
'''

full_query = query

In [2]:
from secret.config import SQL_URL
import requests
import sqlalchemy as sa
import pandas as pd

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(full_query, con=engine)

,oid,probability,candid,fid,mjd,magstat_fid,dmdt_first
0,ZTF18aaiaclp,0.520409,2434223984915015031,2,60188.223982,2,-0.445361
1,ZTF18aakjgzj,0.319556,2434219693515015039,2,60188.219699,2,-0.341342
2,ZTF18aaleydm,0.613368,2434217801715015066,2,60188.217801,2,-0.830610
3,ZTF18aamxief,0.323881,2434223031815015040,2,60188.223032,2,-0.298997
4,ZTF18aanglcr,0.336435,2434215873315015004,2,60188.215880,2,-0.713267
5,ZTF18aaouvke,0.282244,2434220161815015099,2,60188.220162,2,-0.333054
6,ZTF18aaqsuyn,0.414416,2434215875015015115,2,60188.215880,2,-0.880376
7,ZTF18aattenu,0.599858,2434216851815015097,2,60188.216852,2,-0.651601
8,ZTF18aaxkcvg,0.492333,2434224440015015034,2,60188.224444,2,-0.346522
9,ZTF18abbinqq,0.497010,2434182954115015170,1,60188.182951,1,-0.529858
